# CANDIDATES

The candidates file is a csv file that contains nodule candidate per line. Each line holds the scan name, the x, y, and z position of each candidate in world coordinates, and the corresponding class. The list of candidates is provided for participants who are following the ‘false positive reduction’ track. Tutorial on how to view lesions given the location of candidates will be available on the Forum page.

The candidate locations are computed using three existing candidate detection algorithms [1-3]. As lesions can be detected by multiple candidates, those that are located <= 5 mm are merged. Using this method, 1120 out of 1186 nodules are detected with 551,065 false positives. For convenience, the corresponding class label (0 for non-nodule and 1 for nodule) for each candidate is provided in the list. It has to be noted that there can be multiple candidates per nodule.

In [1]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import glob
import fnmatch
import os
import SimpleITK as sitk
import time
from matplotlib.patches import Rectangle
pd.options.display.max_colwidth = 100

In [ ]:
def findFullPaths(candidates):
    #candidates["seriesuidFullPaths"] = "toFind" # New column with new paths to find
    candidates = pd.read_csv("/home/msmith/luna16/CSVFILES/candidates.csv")
    include = ["subset" + str(i) for i in xrange(10)]
    directories = []
    for roots, dirs, files in os.walk('/home/msmith/luna16/',topdown=True):
            dirs[:] = [d for d in dirs if d in include]
            for d in dirs:
                directories.append(os.path.abspath(d))  
    fullPaths = []
    for directory in directories:
        for _,_,files in os.walk(directory):
            for x in files:
                fullPaths.append(os.path.abspath(os.path.join(directory,x)))

    rawPaths = [path for path in fullPaths if ".raw" in path]
    fullPath = lambda seriesuid: [s for s in rawPaths if seriesuid in s][0]
    candidates["seriesuidFullPath"] = candidates.seriesuid.apply(fullPath)
    #candidates.to_csv("/home/msmith/luna16/CSVFILES/candidatesFP.csv")

In [2]:
# Now to find voxel coords
def load_originSpacing(filename):
    itkimage = sitk.ReadImage(filename)
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
    return numpyOrigin, numpySpacing

def worldToVoxelCoord(worldCoord, origin, spacing):
    stretchedVoxelCoord = np.absolute(worldCoord - origin)
    voxelCoord = stretchedVoxelCoord / spacing
    return voxelCoord

def getNVCmhd(row): # Much faster
    mhdPath = row["seriesuidFullPath"].replace(".raw",".mhd")
    with open(mhdPath,"r") as f:
        metaData = f.read()
        metaDataSplit = metaData.split("\n")
        imgOrigin = np.array(metaDataSplit[6].split(" ")[-3:]).astype(float)[::-1]
        imgSpacing = np.array(metaDataSplit[9].split(" ")[-3:]).astype(float)[::-1]

    noduleCoords = row['coordX'],row['coordY'],row['coordZ']
    noduleCoords = noduleCoords[::-1]
    nvc = worldToVoxelCoord(noduleCoords,imgOrigin,imgSpacing)
    nvc = np.array(nvc.astype(int))
    return np.array([nvc, imgSpacing])

def getSpacing(row):
    mhdPath = row["seriesuidFullPath"].replace(".raw",".mhd")
    _, imgSpacing = load_originSpacing(mhdPath)
    return imgSpacing


In [4]:
# Add voxel coords and spacing to original data set
def addVoxSpacing():
    candidates = pd.read_csv("/home/msmith/luna16/CSVFILES/candidatesFP.csv")
    candidates.rename(columns={'class':'Class'}, inplace=True)
    data = candidates
    start = time.clock()
    nObs = data.shape[0]
    xyz = data.apply(lambda row: pd.Series( dict(zip(["z","y","x","spacingZ","spacingY","spacingX"],getNVCmhd(row).flatten()))), axis=1)
    data = data.merge(xyz,left_index=True,right_index=True)
    end = time.clock()
    print("Time taken =", end-start)
    data.to_csv("/home/msmith/luna16/CSVFILES/candidatesFPNVC.csv")
    #data

In [2]:
candidates = pd.read_csv("/home/msmith/luna16/CSVFILES/candidatesFPNVC.csv")
candidatesSub = candidates[["seriesuidFullPath","Class","spacingX","spacingY","spacingZ","x","y","z"]]
#candidatesSub.to_csv("/home/msmith/luna16/CSVFILES/candidatesCleaned.csv",index=0)

In [51]:
def trainTestSplit(data, splitPercentage):
    nObs = data.shape[0]
    idx = np.random.permutation(nObs)
    split = 0.8
    trainIdx, testIdx = np.split(idx,[len(idx)*splitPercentage])
    isDisjoint = np.in1d(trainIdx,testIdx)
    isDisjoint.sum()
    train, test = data.iloc[trainIdx], data.iloc[testIdx]
    print("Train/test class counts")
    print("Train ==> ","\n", train.Class.value_counts(normalize=1))
    print("Test ==> ","\n", test.Class.value_counts(normalize=1))
    return train,test

In [58]:
splitPercentage = 0.8
train, test = trainTestSplit(candidatesSub,splitPercentage)
train.to_csv("/home/msmith/luna16/CSVFILES/candidatesTrain"+str(splitPercentage)[2]+".csv",index=0)
test.to_csv("/home/msmith/luna16/CSVFILES/candidatesTest"+str(splitPercentage)[2]+".csv",index=0)

Train/test class counts
('Train ==> ', '\n', 0    0.997534
1    0.002466
dtype: float64)
('Test ==> ', '\n', 0    0.997605
1    0.002395
dtype: float64)


In [59]:
train

,seriesuidFullPath,Class,spacingX,spacingY,spacingZ,x,y,z
157112,/home/msmith/luna16/subset4/1.3.6.1.4.1.14519.5.2.1.6279.6001.185226274332527104841463955058.raw,0,0.791016,0.791016,1.250,184,266,116
337583,/home/msmith/luna16/subset8/1.3.6.1.4.1.14519.5.2.1.6279.6001.292049618819567427252971059233.raw,0,0.644531,0.644531,2.000,413,321,54
549865,/home/msmith/luna16/subset5/1.3.6.1.4.1.14519.5.2.1.6279.6001.986011151772797848993829243183.raw,0,0.703125,0.703125,2.500,188,261,101
478334,/home/msmith/luna16/subset2/1.3.6.1.4.1.14519.5.2.1.6279.6001.624425075947752229712087113746.raw,0,0.664062,0.664062,1.250,141,271,146
114757,/home/msmith/luna16/subset7/1.3.6.1.4.1.14519.5.2.1.6279.6001.163217526257871051722166468085.raw,0,0.871094,0.871094,1.250,311,249,110
202696,/home/msmith/luna16/subset3/1.3.6.1.4.1.14519.5.2.1.6279.6001.210426531621179400035178209430.raw,0,0.703125,0.703125,2.500,356,275,59
364001,/home/msmith/luna16/subset8/1.3.6.1.4.1.14519.5.2.1.6279.6001.306520140119968755187868602181.raw,0,0.585938,0.585938,2.000,189,298,52
415016,/home/msmith/luna16/subset1/1.3.6.1.4.1.14519.5.2.1.6279.6001.336894364358709782463716339027.raw,0,0.703125,0.703125,0.625,201,264,335
141032,/home/msmith/luna16/subset5/1.3.6.1.4.1.14519.5.2.1.6279.6001.176638348958425792989125209419.raw,0,0.591797,0.591797,1.250,342,368,209
230807,/home/msmith/luna16/subset8/1.3.6.1.4.1.14519.5.2.1.6279.6001.226152078193253087875725735761.raw,0,0.671875,0.671875,1.000,374,181,72


## Display Image

In [6]:
#Get a range in which to subset the data
#data = candidatesSub
data = candidatesSub.loc[(candidates.Class==1)] # Looking at confirmed nodules
keep = np.random.randint(0,data.shape[0],1)
#keep = 11105
data = data.iloc[keep]
#obs = data.iloc[0]
obs = data
nvc = np.array([obs.z,obs.y,obs.x]).astype(int)

mhdPath = obs.seriesuidFullPath.replace(".raw",".mhd")
itkimage = sitk.ReadImage(mhdPath)
img = sitk.GetArrayFromImage(itkimage)

margin = 50
nvcMinus,nvcPlus = (nvc - margin).astype(int), (nvc + margin).astype(int)
def printNoduleInfo():
    print("obs =", obs)
    print("Image shape ==>" + "\n" + str(img.shape))
    print("Nodule coordinates @"+"\n" + str(nvc))
    print("NVC margins ==>" + "\n" + str(nvcMinus) + "\n" + str(nvcPlus))
printNoduleInfo()

from matplotlib.patches import Rectangle
rectangle = plt.Rectangle((nvcMinus[2], nvcMinus[1]),margin*2 ,margin*2, facecolor = 'none', ec='r')
circle = plt.Circle((nvc[2],nvc[1]),10, facecolor = 'none', ec='r')
plt.gca().add_patch(rectangle)
plt.gca().add_patch(circle)
plt.imshow(img[nvc[0]],cmap=cm.gray)
plt.show()

for i in xrange(nvc[0]-2,nvc[0]+2,1):
    if i == nvc[0]:
        print("Nodule located here")
    print(i)
    plt.imshow(img[i,nvcMinus[1]:nvcPlus[1],nvcMinus[2]:nvcPlus[2]],cmap=cm.gray)
    plt.show()

KeyError: 0